<a href="https://colab.research.google.com/github/lucasfabs/cours-deeplearning-dv/blob/main/temp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [24]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import torch.nn as nn

In [25]:
# Cellule supprimée - TensorFlow non nécessaire pour ce projet PyTorch

In [26]:
# Transforms avec Normalize et Compose
# Pour MNIST: moyenne = 0.1307, écart-type = 0.3081
transform = Compose([
    ToTensor(),
    Normalize((0.1307,), (0.3081,))
])

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

In [27]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [28]:
print(training_data.classes)
print(training_data.data.size())

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [29]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      self.max_pool = nn.MaxPool2d(2)
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)
      self.fc1 = nn.Linear(1600, 128)
      self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Bloc de convolution 1
        x = self.conv1(x)
        x = F.relu(x)
        x = self.max_pool(x)
        x = self.dropout1(x)

        # Bloc de convolution 2
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool(x)
        x = self.dropout2(x)

        # Aplatissement et couches fully connected
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x

# Création du modèle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
print(model)

# test
random_data = torch.rand((1, 1, 28, 28)).to(device)
result = model(random_data)
print (result)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
tensor([[ 0.0266,  0.0169,  0.0014,  0.0440, -0.0051, -0.0653, -0.0062,  0.0057,
          0.0673, -0.1317]], grad_fn=<AddmmBackward0>)


In [30]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# Création du SummaryWriter pour TensorBoard
writer = SummaryWriter('runs/mnist_experiment')

In [31]:
def train(dataloader, model, loss_fn, optimizer, epoch, writer):
    size = len(dataloader.dataset)
    model.train()
    running_loss = 0.0
    for batch_idx, batch_value in enumerate(dataloader):
        x, y = batch_value
        x, y = x.to(device), y.to(device)

        # forward pass
        pred = model(x)
        loss = loss_fn(pred, y)

        # backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()
        
        if batch_idx % 100 == 0:
            loss, current = loss.item(), (batch_idx+1) * len(x)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            # Log du loss par batch
            writer.add_scalar('Training Loss/Batch', loss, epoch * len(dataloader) + batch_idx)
    
    # Log de la moyenne du loss pour l'epoch
    avg_loss = running_loss / len(dataloader)
    writer.add_scalar('Training Loss/Epoch', avg_loss, epoch)

In [32]:
def test(dataloader, model, loss_fn, epoch, writer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0 # sur python associe 1 valeur a deux variables d'un coup
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss/=num_batches
    accuracy = correct/size
    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    # Log des métriques de test
    writer.add_scalar('Test Loss', test_loss, epoch)
    writer.add_scalar('Test Accuracy', 100*accuracy, epoch)

In [33]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer, t, writer)
    test(test_dataloader, model, loss_fn, t, writer)
print("Done!")

# Fermeture du writer
writer.close()

Epoch 1
-------------------------------


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


loss: 2.314964 [   64/60000]
loss: 2.285397 [ 6464/60000]
loss: 2.271247 [12864/60000]
loss: 2.262254 [19264/60000]
loss: 2.201883 [25664/60000]
loss: 2.142508 [32064/60000]
loss: 2.134722 [38464/60000]


KeyboardInterrupt: 

In [ ]:
# Visualiser TensorBoard dans Google Colab
# Exécutez cette cellule APRÈS avoir lancé l'entraînement
# Le tableau de bord TensorBoard apparaîtra directement dans cette cellule
%tensorboard --logdir runs
